In [3]:
#!/usr/bin/env python3
"""
controller.py
Real-time solar-aware controller for the water heater.

Functions:
1) Load the daily schedule triac_schedule.csv (Time_minute, Boiler_kW_min, TRIAC_duty).
2) Download hourly spot prices (SE3) and attach per-minute prices.
3) Every dt_s seconds:
   - Read export_kW from ntmm.org/pqlog/data/YYYY-MM-DD.txt
   - Read tank_soc (currently dummy)
   - Run realtime_controller_step()
   - Send TRIAC duty command (currently print)
   - Update energy credit and perform reallocation every 10 minutes
"""

import time
from dataclasses import dataclass
from datetime import datetime
import requests
import pandas as pd
import pytz

# ------------------------- CONSTANTS -------------------------
P_MAX = 2.2  # kW – Maximum heater power
TZ = pytz.timezone("Europe/Stockholm")

# Path to your schedule (generated by the optimization script)
SCHEDULE_CSV = "/Users/iraklisbournazos/Desktop/Nathaniel Taylor project/Hadrien scripts/triac_schedule.csv"

# Log file for debugging / analysis
LOG_CSV = "controller_log.csv"

ZONE = "SE3"   # Nordpool price zone

# Meter export log location (real-time PV export from NTMM)
PQLOG_BASE_URL = "https://ntmm.org/pqlog/data/"
EXPORT_COL_INDEX = 6  # Column 7 = export in Watts


# ------------------------- PARAMETERS & STATE -------------------------
@dataclass
class RTParams:
    dt_s: float = 10.0          # Controller time-step (seconds)
    export_thr_on: float = 0.15 # kW – threshold to start boosting
    export_thr_off: float = 0.05# kW – threshold to stop boosting
    soc_min: float = 0.1        # minimum water tank SOC to allow boosting
    soc_margin: float = 0.05
    credit_thr_kWh: float = 0.3 # threshold for cutting future energy


@dataclass
class RTState:
    E_opt_cum: float = 0.0      # cumulative *scheduled* energy (kWh)
    E_act_cum: float = 0.0      # cumulative *actual* energy (kWh)
    E_credit: float = 0.0       # difference actual - scheduled
    boosting: bool = False      # whether boosting is active


# ------------------------- PRICE FETCHING -------------------------
def fetch_spot_prices_for_date(date, zone=ZONE):
    """
    Fetches hourly electricity prices for a specific date.
    Source: https://www.elprisetjustnu.se/
    """
    date_str = date.strftime("%Y/%m-%d")
    url = f"https://www.elprisetjustnu.se/api/v1/prices/{date_str}_{zone}.json"
    resp = requests.get(url)
    resp.raise_for_status()
    data = resp.json()

    records = []
    for hour in data:
        records.append({
            "Time": hour["time_start"],
            "Price_SEK_kWh": hour["SEK_per_kWh"],
            "Price_EUR_kWh": hour["EUR_per_kWh"],
        })

    df = pd.DataFrame(records)
    df["Time"] = pd.to_datetime(df["Time"])
    return df.sort_values("Time").reset_index(drop=True)


def attach_price_to_triac_df(triac_df, zone=ZONE):
    """
    Adds per-minute electricity price to the TRIAC schedule.
    Converts hourly prices → per-minute using forward-fill.
    """
    if "Price_SEK_kWh" in triac_df.columns:
        return triac_df  # Already included

    triac_df = triac_df.copy()
    triac_df["Time_minute"] = pd.to_datetime(triac_df["Time_minute"])
    triac_df.sort_values("Time_minute", inplace=True)

    schedule_date = triac_df["Time_minute"].iloc[0].date()
    df_hourly = fetch_spot_prices_for_date(schedule_date, zone)
    df_hourly = df_hourly.set_index("Time")

    df_min = df_hourly["Price_SEK_kWh"].resample("1min").ffill().to_frame()

    triac_df = triac_df.set_index("Time_minute")
    triac_df = triac_df.join(df_min, how="left")
    return triac_df.reset_index()


# ------------------------- REAL-TIME CONTROL -------------------------
def realtime_controller_step(now, export_kW, tank_soc, triac_df, params, state):
    """
    Computes TRIAC duty based on:
    - scheduled baseline power
    - available solar export
    - tank SOC
    - boosting hysteresis
    """
    now_ts = pd.Timestamp(now)
    current_minute = now_ts.floor('min')

    if current_minute not in triac_df.index:
        return 0.0, state  # Outside schedule → OFF

    P_base = float(triac_df.loc[current_minute, "Boiler_kW_min"])

    # Scheduled baseline energy accumulation
    dt_h = params.dt_s / 3600.0
    state.E_opt_cum += P_base * dt_h

    # Boosting hysteresis logic
    if state.boosting:
        if export_kW < params.export_thr_off:
            state.boosting = False
    else:
        if export_kW > params.export_thr_on:
            state.boosting = True

    # Compute commanded power
    if state.boosting and tank_soc > params.soc_min:
        P_room = P_MAX - P_base
        P_cmd = P_base + min(export_kW, max(P_room, 0))
    else:
        P_cmd = P_base

    P_cmd = max(0, min(P_cmd, P_MAX))

    # Actual energy accumulation
    state.E_act_cum += P_cmd * dt_h
    state.E_credit = state.E_act_cum - state.E_opt_cum

    duty = max(0.0, min(P_cmd / P_MAX, 1.0))
    return duty, state


# ------------------------- REALLOCATION -------------------------
def reallocate_future_schedule(now, triac_df, price_series, state, params, soc_current):
    """
    Removes or reduces future minutes starting from the most expensive ones.
    Triggered when E_credit > credit_thr_kWh.
    """
    if state.E_credit < params.credit_thr_kWh:
        return
    if soc_current < params.soc_min + params.soc_margin:
        return

    now_ts = pd.Timestamp(now)
    future_minutes = triac_df.index[triac_df.index > now_ts.floor("min")]
    if len(future_minutes) == 0:
        return

    future_prices = price_series.loc[future_minutes]
    sorted_minutes = future_prices.sort_values(ascending=False).index

    E_to_cut = state.E_credit

    for t in sorted_minutes:
        if E_to_cut <= 0:
            break

        P_base = float(triac_df.at[t, "Boiler_kW_min"])
        if P_base <= 0:
            continue

        E_slot = P_base * (1/60)  # kWh per minute

        if E_to_cut >= E_slot:
            triac_df.at[t, "Boiler_kW_min"] = 0
            triac_df.at[t, "TRIAC_duty"] = 0
            E_to_cut -= E_slot
        else:
            P_new = P_base - E_to_cut * 60
            triac_df.at[t, "Boiler_kW_min"] = P_new
            triac_df.at[t, "TRIAC_duty"] = P_new / P_MAX
            E_to_cut = 0

    state.E_credit = E_to_cut


# ------------------------- SCHEDULE LOADING -------------------------
def load_schedule(csv_path):
    triac_df = pd.read_csv(csv_path, parse_dates=["Time_minute"])
    triac_df = attach_price_to_triac_df(triac_df, zone=ZONE)
    triac_df.set_index("Time_minute", inplace=True)
    return triac_df, triac_df["Price_SEK_kWh"].copy()


# ------------------------- METER READING -------------------------
def get_export_from_meter():
    """
    Fetches the LATEST export measurement from NTMM.
    Returns export power in kW.
    """
    today = datetime.now(TZ).date()
    fname = today.strftime("%Y-%m-%d.txt")
    url = PQLOG_BASE_URL + fname

    try:
        resp = requests.get(url, timeout=3)
        resp.raise_for_status()
    except:
        return 0.0

    lines = resp.text.strip().splitlines()
    if not lines:
        return 0.0

    last_line = lines[-1]
    parts = last_line.split()

    try:
        P_export_W = float(parts[EXPORT_COL_INDEX])
    except:
        return 0.0

    return max(0, P_export_W / 1000)


def get_tank_soc():
    """
    Placeholder — eventually replaced by a real sensor or estimator.
    """
    return 0.5


def send_duty_to_triac(duty):
    """
    Sends command to TRIAC.
    For now: print.
    On Raspberry Pi: replace with GPIO / serial output.
    """
    pwm_value = int(duty * 255)
    print(f"[CMD] duty={duty:.3f}, pwm={pwm_value}")


# ------------------------- LOGGING -------------------------
def log_step(log_list, now, duty, export_kW, tank_soc, state):
    log_list.append({
        "Time": now,
        "Duty": duty,
        "Export_kW": export_kW,
        "Tank_SOC": tank_soc,
        "E_opt_cum": state.E_opt_cum,
        "E_act_cum": state.E_act_cum,
        "E_credit": state.E_credit,
    })


# ------------------------- MAIN LOOP -------------------------
def main():
    params = RTParams()
    state = RTState()

    triac_df, price_series = load_schedule(SCHEDULE_CSV)
    logs = []

    print("Starting controller loop...")
    while True:
        now = datetime.now(TZ)

        # Check schedule date
        if now.date() != triac_df.index[0].date():
            print("Outside schedule day → boiler OFF")
            send_duty_to_triac(0)
            time.sleep(params.dt_s)
            continue

        # Real-time readings
        export_kW = get_export_from_meter()
        tank_soc = get_tank_soc()

        # Compute duty
        duty, state = realtime_controller_step(
            now, export_kW, tank_soc, triac_df, params, state
        )

        send_duty_to_triac(duty)
        log_step(logs, now, duty, export_kW, tank_soc, state)

        # Reallocation every 10 minutes
        if now.minute % 10 == 0 and now.second < params.dt_s:
            reallocate_future_schedule(
                now, triac_df, price_series, state, params, soc_current=tank_soc
            )

        # Save logs every hour
        if now.minute == 59 and now.second < params.dt_s:
            pd.DataFrame(logs).to_csv(LOG_CSV, index=False)
            print("Log saved to", LOG_CSV)

        time.sleep(params.dt_s)


if __name__ == "__main__":
    main()




Starting controller loop...
[CMD] duty=0.000, pwm=0
[CMD] duty=0.000, pwm=0
[CMD] duty=0.000, pwm=0
[CMD] duty=1.000, pwm=255


KeyboardInterrupt: 